In [34]:
# coding: utf-8

# In[38]:


from __future__ import print_function, division
import requests
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
from bs4 import BeautifulSoup
from html.parser import HTMLParser
from selenium import webdriver
import selenium
from selenium.webdriver.common.keys import Keys
import time
import os
import csv

home_page = 'https://www.edmunds.com'

# response = requests.get(url)
# page = response.text

chromedriver = "/usr/local/bin/chromedriver"  # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)
driver.get(home_page)
html = driver.page_source


# In[39]:


# response.status_code
# print(response.text)
# soup = BeautifulSoup(driver.page_source, 'lxml')


# In[40]:



In [35]:

def get_list_of_make():
    try:
        select_box = driver.find_element_by_name("select-make")
    except(selenium.common.exceptions.InvalidSelectorException, selenium.common.exceptions.NoSuchElementException):
        pass
    # this part searches the elements contained inside of select_box
    # and then adds them to the list options if they have the tag name "options"
    options = [x for x in select_box.find_elements_by_tag_name("option")]
    template = '//select[@name="select-make"]/option[@value="{}"]'
    # for element in options:
    #    print(element.get_attribute("value"))
    count = 0
    listofvalues = []
    for element in options:
        count = count + 1
        index = element.get_attribute("value")
        templateoption = template.format(index)
        if index:
            listofvalues.append(templateoption)
    # if(count >= 3):
    #    break
    print(listofvalues)
    return listofvalues


# In[41]:


def get_list_of_model(make):
    driver.find_element_by_xpath(make).click()
    soup = BeautifulSoup(driver.page_source, 'lxml')
    time.sleep(1)
    try:
        select_model_box = driver.find_element_by_name("select-model")
    except(selenium.common.exceptions.InvalidSelectorException, selenium.common.exceptions.NoSuchElementException):
        return []
    options = [x for x in select_model_box.find_elements_by_tag_name("option")]

    template = '//select[@name="select-model"]/option[@value="{}"]'
    # for element in options:
    #    print(element.get_attribute("value"))

    count = 0
    listofmodels = []
    for element in options:
        count = count + 1
        index = element.get_attribute("value")
        templateoption = template.format(index)
        # print(templateoption)
        if index:
            listofmodels.append(templateoption)
        # if(count >= 3):
        #    break
    print(listofmodels)
    return listofmodels


# In[42]:


# get_list_of_model('//select[@name="select-make"]/option[@value="bentley"]')


# In[43]:


def get_car_spec_url(make, model):
    driver.find_element_by_xpath(make).click()
    soup = BeautifulSoup(driver.page_source, 'lxml')
    time.sleep(1)
    driver.find_element_by_xpath(model).click()
    soup = BeautifulSoup(driver.page_source, 'lxml')
    time.sleep(1)
    new_car_select = '//select[@name="select-pubstate"]/option[@value="NEW"]'
    driver.find_element_by_xpath(new_car_select).click()
    go_button = driver.find_element_by_link_text('GO')
    go_button.click()
    features_and_specs_link = driver.find_element_by_partial_link_text(' features & specs')
    features_and_specs_link.click()
    # print(driver.current_url)
    return driver.current_url


# In[44]:


def tuple_output_heading_extractor(soup):
    datarows = []
    tablerows = soup.find_all('table')
    #table_bodyrows = tablerows.find('tbody')

    rows_heading = [row for row in tablerows[3].find_all('tr')]
    for row in rows_heading:
        cols_head = row.find_all('th')
        if not cols_head:
            continue
        cols_head = [ele.text.strip() for ele in cols_head]
        datarows.append([ele for ele in cols_head if ele])
    #datarows.append("price")  # Get rid of empty values
    #datarows.append("car_model")
    stringoutput = [''.join(x) for x in datarows[1:]]
    # print(stringoutput)
    tupleoutput = tuple(stringoutput)
    return tupleoutput


def table_data_extractor(soup):
    data = []
    tablerows = soup.find_all('table')
    #table_body = tablerows.find('tbody')

    rows = [row for row in tablerows[3].find_all('tr')]
    #print(rows)
    for row in rows:
        cols = row.find_all('td')
        if not cols:
            continue
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])  # Get rid of empty values
        lst2 = [item[0] for item in data]
    return (lst2)


def getDataFrameForFeatureSpecs(url):
    driver.get(url)
    html = driver.page_source
    # Alternatives to Chrome: Firefox, phantomjs
    # var = response.status_code
    # print(response.text)
    soup = BeautifulSoup(driver.page_source, 'lxml')

    tuple_output_heading_extractor(soup)

    try:
        select_box = driver.find_element_by_name("core_page_view_content_features")
    except(selenium.common.exceptions.InvalidSelectorException, selenium.common.exceptions.NoSuchElementException):
        return {}
    # if your select_box has a name.. why use xpath?

    # this part searches the elements contained inside of select_box
    # and then adds them to the list options if they have the tag name "options"
    options = [x for x in select_box.find_elements_by_tag_name("option")]

    template = '//select[@name="core_page_view_content_features"]/option[@value="{}"]'
    # for element in options:
    #    print(element.get_attribute("value"))

    listofvalues = []
    for element in options:
        index = element.get_attribute("value")
        templateoption = template.format(index)
        listofvalues.append(templateoption)
        # or append to list or whatever you want here

    dict_of_metrics = {}
    list_of_values_and_header = {}
    for i in listofvalues:
        driver.find_element_by_xpath(i).click()
        soup = BeautifulSoup(driver.page_source, 'lxml')
        time.sleep(1)
        header_of_metrics = tuple_output_heading_extractor(soup)
        list_of_metric_values = table_data_extractor(soup)
        name_of_car = driver.find_element_by_xpath(i).text
        #title = soup.find('title').text
        #price = '//span[(@class= "display-3 font-weight-bold")]'
        #price_of_car = driver.find_element_by_xpath(price).text
        #list_of_metric_values.append(price_of_car)
        #list_of_metric_values.append(title)

        # create a dict from two values
        list_of_values_and_header = dict(zip(header_of_metrics, list_of_metric_values))
        dict_of_metrics[name_of_car] = list_of_values_and_header
    # metrics_dataset: DataFrame = pd.DataFrame.from_dict(dict_of_metrics, orient='index',columns=tupleoutput)
    # print(metrics_dataset)

    return dict_of_metrics


# url = 'https://www.edmunds.com/ford/f-250-super-duty/2017/supercab/features-specs/'
# df = getDataFrameForFeatureSpecs(url)
# print(df)


# In[ ]:


make_list = get_list_of_make()
# driver.get(home_page)
df = {}
for each_make in make_list:
    driver.get(home_page)
    model_list = get_list_of_model(each_make)
    for each_model in model_list:
        driver.get(home_page)
        time.sleep(1)
        try:
            url = get_car_spec_url(each_make, each_model)
        except(selenium.common.exceptions.InvalidSelectorException, selenium.common.exceptions.NoSuchElementException):
            continue
        df1 = getDataFrameForFeatureSpecs(url)

        print(url)
        driver.get(home_page)
        if bool(df1):
            df.update(df1)

# In[1]:


['//select[@name="select-make"]/option[@value="acura"]', '//select[@name="select-make"]/option[@value="alfa-romeo"]', '//select[@name="select-make"]/option[@value="aston-martin"]', '//select[@name="select-make"]/option[@value="audi"]', '//select[@name="select-make"]/option[@value="bentley"]', '//select[@name="select-make"]/option[@value="bmw"]', '//select[@name="select-make"]/option[@value="buick"]', '//select[@name="select-make"]/option[@value="cadillac"]', '//select[@name="select-make"]/option[@value="chevrolet"]', '//select[@name="select-make"]/option[@value="chrysler"]', '//select[@name="select-make"]/option[@value="dodge"]', '//select[@name="select-make"]/option[@value="ferrari"]', '//select[@name="select-make"]/option[@value="fiat"]', '//select[@name="select-make"]/option[@value="ford"]', '//select[@name="select-make"]/option[@value="genesis"]', '//select[@name="select-make"]/option[@value="gmc"]', '//select[@name="select-make"]/option[@value="honda"]', '//select[@name="select-ma

IndexError: list index out of range

In [36]:
print(df)

#with open('test.csv', 'w') as f:
 #   for key in df.keys():
  #      f.write("%s,%s\n" % (key, df[key]))

pandas_data_frame = pd.DataFrame(df)
#pandas_data_frame.to_csv('dataframe.csv')

pandas_data = pandas_data_frame.T
pandas_data.rename_axis('car_make_model')
#pandas_data.to_csv('safety_features_head_list.csv')
pandas_data.to_csv('addl_features_head_list.csv')


{'4dr Sedan (2.4L 4cyl 8AM)': {'Torque': '182 ft-lbs. @ 4500 rpm', 'Base engine size': '2.4 L', 'Horsepower': '206 hp @ 6800 rpm', 'Turning circle': '39.2 ft.', 'Valves': '16', 'direct injection': 'yes', 'Base engine type': 'gas', 'Valve timing': 'Variable', 'Cam type': 'Double overhead cam (DOHC)', 'Cylinders': 'inline 4', 'cylinder deactivation': 'no'}, 'Special Edition 4dr Sedan (2.4L 4cyl 8AM)': {'Torque': '180 ft-lbs. @ 3800 rpm', 'Base engine size': '2.4 L', 'Horsepower': '201 hp @ 6800 rpm', 'Turning circle': '36.8 ft.', 'Valves': '16', 'direct injection': 'yes', 'Base engine type': 'gas', 'Valve timing': 'Variable', 'Cam type': 'Double overhead cam (DOHC)', 'Cylinders': 'inline 4'}, 'Premium Package 4dr Sedan (2.4L 4cyl 8AM)': {'Torque': '180 ft-lbs. @ 3800 rpm', 'Base engine size': '2.4 L', 'Horsepower': '201 hp @ 6800 rpm', 'Turning circle': '36.8 ft.', 'Valves': '16', 'direct injection': 'yes', 'Base engine type': 'gas', 'Valve timing': 'Variable', 'Cam type': 'Double overhe